In [3]:
print("OK")

OK


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [5]:
#extract data from the pdf
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [8]:
exracted_data=load_pdf(r"C:\Users\91939\Desktop\Medicalchatbot-with-llama2\data")

In [4]:
#exracted_data

In [9]:
#create text chunks
def text_split(exracted_data):
    text_spilter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_spilter.split_documents(exracted_data)

    return text_chunks

In [ ]:
text_chunks=text_split(exracted_data)
print(len(text_chunks))

In [ ]:
#download embedding model
def download_hugging_face_embeddings():
   embeddings = HuggingFaceEmbeddings(model_name="sentence-transformer/all-MiniLM-L6-v2")
   
   return embeddings


In [ ]:
embeddings= download_hugging_face_embeddings()


In [ ]:
#!pip install --upgrade huggingface_hub sentence-transformers transformers


In [ ]:
query_res=embeddings.embed_query("Hello World")
print("lenght",len(query_res))

In [ ]:
pinecone.init(api_key=os.environ.get(Pinecone_api_key))

index_name="medicochatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
# Import the Pinecone library
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=Pinecone_api_key)

# Define a sample dataset where each item has a unique ID and piece of text
data = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
    {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
]

# Convert the text into numerical vectors that Pinecone can index
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data],
    parameters={"input_type": "passage", "truncate": "END"}
)

print(embeddings)
